In [1]:
import os
import pandas as pd
from google_drive_downloader import GoogleDriveDownloader as gdd

In [2]:
## download training data from google drive
gdd.download_file_from_google_drive(file_id='1nRDgiaeP8r6Mh2GsY6AUFZIwCmSoQWXB',
                                    dest_path='./data.xlsx',
                                    overwrite=True)

In [3]:
## loads excel
xls = pd.ExcelFile('./data.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]

In [4]:
# !pip install hunspell-ko

# from hanspell import spell_checker

import requests
import json
import sys
import xml.etree.ElementTree as ET

base_url = 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'

_agent = requests.Session()
PY3 = sys.version_info[0] == 3


def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>', '')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):

    if isinstance(text, list):
        print('yes')
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result

    payload = {
        '_callback': 'window.__jindo2_callback._spellingCheck_0',
        'q': text
    }

    headers = {
        'user-agent': 'https://github.com/michael960725',
        'referer': 'https://naver.com/',
    }

    r = _agent.get(base_url, params=payload, headers=headers)
    r = r.text[42:-2]

    data = json.loads(r)
    html = data['message']['result']['html']
    return _remove_tags(html)

# for i in data_without_ground_truth:
#   # print(i)
#   print(check(i))
